In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import partial
%matplotlib inline

In [2]:
df_full = pd.read_csv('../data/LoanStats2014.csv', skiprows=1, skipfooter=2)

/home/ubuntu/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [3]:
df = df_full.copy()

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235629 entries, 0 to 235628
Columns: 111 entries, id to total_il_high_credit_limit
dtypes: float64(47), int64(41), object(23)
memory usage: 199.5+ MB


In [5]:
df_full.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,NaN,NaN,10400,10400,10400,36 months,6.99%,321.08,A,A3,...,0,4,83.3,14.3,0,0,179407,15030,13000,11325
1,NaN,NaN,15000,15000,15000,60 months,12.39%,336.64,C,C1,...,0,4,100.0,0.0,0,0,196500,149140,10000,12000
2,NaN,NaN,21425,21425,21425,60 months,15.59%,516.36,D,D1,...,0,2,91.4,100.0,0,0,57073,42315,15000,35573
3,NaN,NaN,7650,7650,7650,36 months,13.66%,260.20,C,C3,...,0,2,100.0,100.0,0,0,82331,64426,4900,64031
4,NaN,NaN,12800,12800,12800,60 months,17.14%,319.08,D,D4,...,0,0,76.9,100.0,0,0,368700,18007,4400,18000


### Define targets

</br> Initially, try a binary target... <br />1 if total payment received is >= 1.05% of amount funded<br />0 if loan status is 'charged off', 'default', '31-120 days late', or is fully paid, but not >= 1.05% of funded

In [6]:
def final_status(status, funded_amnt, total_pymnt):
    #define list to drop
    drop_status = ['In Grace Period', 'Late (16-30 days)']
    neg_status = ['Late (31-120 days)', 'Charged Off', 'Default']
    
    if status in drop_status:
        return 99
    elif status in neg_status:
        return 0
    elif status == 'Fully Paid':
        if total_pymnt>funded_amnt*1.05:
            return 1
        else:
            return 0
    elif status == 'Current':
        if total_pymnt>funded_amnt*1.05:
            return 1
        else:
            return 99     

In [7]:
df['target'] = df.apply(lambda x: final_status(x['loan_status'], x['funded_amnt_inv'], x['total_pymnt_inv']), axis=1)

In [8]:
df['target'].value_counts()

1     123417
99     67215
0      44997
Name: target, dtype: int64

In [9]:
df.drop(df[df['target']==99].index, inplace=True)

In [10]:
#df.reset_index(inplace=True, drop=True)

In [11]:
#df.set_index(['id'], inplace=True)

In [12]:
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,target
0,NaN,NaN,10400,10400,10400,36 months,6.99%,321.08,A,A3,...,4,83.3,14.3,0,0,179407,15030,13000,11325,0
1,NaN,NaN,15000,15000,15000,60 months,12.39%,336.64,C,C1,...,4,100.0,0.0,0,0,196500,149140,10000,12000,1
2,NaN,NaN,21425,21425,21425,60 months,15.59%,516.36,D,D1,...,2,91.4,100.0,0,0,57073,42315,15000,35573,1
3,NaN,NaN,7650,7650,7650,36 months,13.66%,260.20,C,C3,...,2,100.0,100.0,0,0,82331,64426,4900,64031,0
5,NaN,NaN,9600,9600,9600,36 months,13.66%,326.53,C,C3,...,3,100.0,60.0,0,0,52490,38566,21100,24890,0


In [13]:
df['id'].describe()

count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: id, dtype: float64

## Clean Data

In [14]:
n_cols = df.shape[1]

In [15]:
drop_cols = [
        'url'
        , 'application_type'
        , 'policy_code'
    ]

In [16]:
df.drop(drop_cols, axis=1, inplace=True)

In [17]:
df.dropna(how='all', axis=1, inplace=True)

### Choose only columns we will have information for when loan originates

In [18]:
for i in df.columns:
    print (", '{}'".format(i))

, 'loan_amnt'
, 'funded_amnt'
, 'funded_amnt_inv'
, 'term'
, 'int_rate'
, 'installment'
, 'grade'
, 'sub_grade'
, 'emp_title'
, 'emp_length'
, 'home_ownership'
, 'annual_inc'
, 'verification_status'
, 'issue_d'
, 'loan_status'
, 'pymnt_plan'
, 'desc'
, 'purpose'
, 'title'
, 'zip_code'
, 'addr_state'
, 'dti'
, 'delinq_2yrs'
, 'earliest_cr_line'
, 'inq_last_6mths'
, 'mths_since_last_delinq'
, 'mths_since_last_record'
, 'open_acc'
, 'pub_rec'
, 'revol_bal'
, 'revol_util'
, 'total_acc'
, 'initial_list_status'
, 'out_prncp'
, 'out_prncp_inv'
, 'total_pymnt'
, 'total_pymnt_inv'
, 'total_rec_prncp'
, 'total_rec_int'
, 'total_rec_late_fee'
, 'recoveries'
, 'collection_recovery_fee'
, 'last_pymnt_d'
, 'last_pymnt_amnt'
, 'next_pymnt_d'
, 'last_credit_pull_d'
, 'collections_12_mths_ex_med'
, 'mths_since_last_major_derog'
, 'acc_now_delinq'
, 'tot_coll_amt'
, 'tot_cur_bal'
, 'total_rev_hi_lim'
, 'acc_open_past_24mths'
, 'avg_cur_bal'
, 'bc_open_to_buy'
, 'bc_util'
, 'chargeoff_within_12_mths'
, '

In [19]:
drop_cols = [
            'funded_amnt'
            , 'funded_amnt_inv'
            , 'emp_title'
            , 'issue_d'
            , 'loan_status'
            , 'pymnt_plan'
            , 'title'
            , 'zip_code'
            , 'addr_state'
            , 'initial_list_status'
            , 'out_prncp'
            , 'out_prncp_inv'
            , 'total_pymnt'
            , 'total_pymnt_inv'
            , 'total_rec_prncp'
            , 'total_rec_int'
            , 'total_rec_late_fee'
            , 'recoveries'
            , 'collection_recovery_fee'
            , 'last_pymnt_d'
            , 'last_pymnt_amnt'
            , 'next_pymnt_d'
            , 'last_credit_pull_d'
            , 'tot_coll_amt'
            , 'tot_cur_bal'
            ]

In [20]:
df.drop(drop_cols, axis=1, inplace=True)

### Turn into correct data type

In [21]:
df.select_dtypes(exclude=[np.number]).T

,0,1,2,3,5,6,7,8,9,10,...,235617,235618,235619,235620,235621,235622,235623,235625,235627,235628
term,36 months,60 months,60 months,36 months,36 months,36 months,36 months,60 months,36 months,36 months,...,36 months,36 months,36 months,60 months,36 months,36 months,36 months,60 months,36 months,36 months
int_rate,6.99%,12.39%,15.59%,13.66%,13.66%,11.99%,11.44%,11.44%,13.66%,17.86%,...,10.99%,11.99%,15.61%,18.25%,7.90%,13.53%,18.25%,19.97%,7.90%,19.20%
grade,A,C,D,C,C,B,B,B,C,D,...,B,B,C,D,A,B,D,D,A,D
sub_grade,A3,C1,D1,C3,C3,B5,B4,B4,C3,D5,...,B2,B3,C4,D3,A4,B5,D3,D5,A4,D3
emp_length,8 years,10+ years,6 years,< 1 year,10+ years,< 1 year,2 years,6 years,10+ years,10+ years,...,n/a,8 years,7 years,5 years,3 years,4 years,10+ years,10+ years,3 years,10+ years
home_ownership,MORTGAGE,RENT,RENT,RENT,RENT,MORTGAGE,RENT,OWN,MORTGAGE,RENT,...,MORTGAGE,RENT,RENT,MORTGAGE,RENT,RENT,RENT,MORTGAGE,OWN,MORTGAGE
verification_status,Not Verified,Source Verified,Source Verified,Source Verified,Source Verified,Source Verified,Not Verified,Verified,Verified,Source Verified,...,Verified,Source Verified,Verified,Verified,Verified,Source Verified,Source Verified,Verified,Verified,Verified
desc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Borrower added on 12/10/13 > all my loans in...,Borrower added on 12/10/13 > pay off higher ...,NaN,NaN,Borrower added on 12/09/13 > consolidate all...,NaN,NaN,NaN,NaN,Borrower added on 12/04/13 > I will like a l...
purpose,credit_card,debt_consolidation,credit_card,debt_consolidation,debt_consolidation,home_improvement,debt_consolidation,debt_consolidation,debt_consolidation,house,...,credit_card,credit_card,credit_card,medical,debt_consolidation,credit_card,vacation,debt_consolidation,credit_card,other
earliest_cr_line,Sep-1989,Aug-1994,Aug-2003,Aug-2002,Nov-1992,Oct-2001,May-2009,Nov-2003,Jan-2001,Jan-1999,...,May-1990,Aug-1983,Nov-1993,Nov-1994,Jul-1998,Aug-1999,Jun-2002,Jun-1997,Feb-2003,Feb-2000


In [22]:
for i in df.select_dtypes(exclude=[np.number]).columns:
    print (", '{0}'".format(i))

, 'term'
, 'int_rate'
, 'grade'
, 'sub_grade'
, 'emp_length'
, 'home_ownership'
, 'verification_status'
, 'desc'
, 'purpose'
, 'earliest_cr_line'
, 'revol_util'


#### Object to numeric

In [23]:
import re

In [24]:
obj_num_cols = [
        'term'
        , 'int_rate'
        , 'emp_length'
        , 'revol_util'
        ]

In [25]:
df['term'] = df['term'].apply(lambda x: re.sub("[^0-9.,]", '', x))
df['term'] = pd.to_numeric(df['term'])

In [26]:
df['int_rate'] = df['int_rate'].apply(lambda x: re.sub("[^0-9.,]", '', x))
df['int_rate'] = pd.to_numeric(df['int_rate'])

In [27]:
df['emp_length'] = ['0' if x=='< 1 year' else x for x in df['emp_length']]
df['emp_length'] = df['emp_length'].apply(lambda x: re.sub("[^0-9.,]", '', x))
df['emp_length'] = pd.to_numeric(df['emp_length'])#, errors=coerce)

In [28]:
df['revol_util'] = df['revol_util'].apply(lambda x: re.sub("[^0-9.,]", '', str(x)))
df['revol_util'] = pd.to_numeric(df['revol_util'])#, errors=coerce)

#### Object to date

In [29]:
obj_date_cols = [
    #'issue_d'
    'earliest_cr_line'
    ]

In [30]:
df[obj_date_cols] = df[obj_date_cols].apply(lambda x: pd.to_datetime(x))

#### Numeric columns

In [31]:
for i in df.select_dtypes(include=[np.number]).columns:
    print (", '{0}'".format(i))

, 'loan_amnt'
, 'term'
, 'int_rate'
, 'installment'
, 'emp_length'
, 'annual_inc'
, 'dti'
, 'delinq_2yrs'
, 'inq_last_6mths'
, 'mths_since_last_delinq'
, 'mths_since_last_record'
, 'open_acc'
, 'pub_rec'
, 'revol_bal'
, 'revol_util'
, 'total_acc'
, 'collections_12_mths_ex_med'
, 'mths_since_last_major_derog'
, 'acc_now_delinq'
, 'total_rev_hi_lim'
, 'acc_open_past_24mths'
, 'avg_cur_bal'
, 'bc_open_to_buy'
, 'bc_util'
, 'chargeoff_within_12_mths'
, 'delinq_amnt'
, 'mo_sin_old_il_acct'
, 'mo_sin_old_rev_tl_op'
, 'mo_sin_rcnt_rev_tl_op'
, 'mo_sin_rcnt_tl'
, 'mort_acc'
, 'mths_since_recent_bc'
, 'mths_since_recent_bc_dlq'
, 'mths_since_recent_inq'
, 'mths_since_recent_revol_delinq'
, 'num_accts_ever_120_pd'
, 'num_actv_bc_tl'
, 'num_actv_rev_tl'
, 'num_bc_sats'
, 'num_bc_tl'
, 'num_il_tl'
, 'num_op_rev_tl'
, 'num_rev_accts'
, 'num_rev_tl_bal_gt_0'
, 'num_sats'
, 'num_tl_120dpd_2m'
, 'num_tl_30dpd'
, 'num_tl_90g_dpd_24m'
, 'num_tl_op_past_12m'
, 'pct_tl_nvr_dlq'
, 'percent_bc_gt_75'
, 'pub

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 168414 entries, 0 to 235628
Data columns (total 65 columns):
loan_amnt                         168414 non-null int64
term                              168414 non-null int64
int_rate                          168414 non-null float64
installment                       168414 non-null float64
grade                             168414 non-null object
sub_grade                         168414 non-null object
emp_length                        159733 non-null float64
home_ownership                    168414 non-null object
annual_inc                        168414 non-null float64
verification_status               168414 non-null object
desc                              13685 non-null object
purpose                           168414 non-null object
dti                               168414 non-null float64
delinq_2yrs                       168414 non-null int64
earliest_cr_line                  168414 non-null datetime64[ns]
inq_last_6mths           

### Datetime to feature

In [33]:
pd.to_datetime('2014-12-01')

Timestamp('2014-12-01 00:00:00')

In [34]:
df['mo_earliest_credit'] = pd.to_datetime('2014-12-01') - df['earliest_cr_line']

In [35]:
df['mo_earliest_credit'] = df['mo_earliest_credit'].apply(lambda x: x.days/365)

In [36]:
df.drop('earliest_cr_line', axis=1, inplace=True)

### Dummies for object cols

Turn description into a bin for now

In [37]:
df['desc_bin'] = [1 if x else 0 for x in df['desc']]

In [38]:
df.drop(['desc'], axis=1, inplace=True)

In [39]:
obj_cols = df.select_dtypes(include=[object]).columns

In [40]:
df[obj_cols].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 168414 entries, 0 to 235628
Data columns (total 5 columns):
grade                  168414 non-null object
sub_grade              168414 non-null object
home_ownership         168414 non-null object
verification_status    168414 non-null object
purpose                168414 non-null object
dtypes: object(5)
memory usage: 7.7+ MB


In [41]:
df['grade'].value_counts()

C    47458
B    45918
D    29785
A    24631
E    14439
F     4890
G     1293
Name: grade, dtype: int64

In [42]:
df['sub_grade_number'] = df['sub_grade'].str[1]

In [43]:
df['sub_grade_number'].value_counts()

4    34363
1    34079
3    33500
5    33333
2    33139
Name: sub_grade_number, dtype: int64

In [44]:
df['home_ownership'].value_counts()

MORTGAGE    83975
RENT        68062
OWN         16376
ANY             1
Name: home_ownership, dtype: int64

In [45]:
df['verification_status'].value_counts()

Source Verified    67346
Not Verified       52303
Verified           48765
Name: verification_status, dtype: int64

In [46]:
df['purpose'].value_counts()

debt_consolidation    102322
credit_card            38169
home_improvement        9430
other                   7912
major_purchase          2824
medical                 1845
small_business          1790
car                     1359
moving                  1080
vacation                 965
house                    610
renewable_energy         103
wedding                    5
Name: purpose, dtype: int64

Don't use emp_title (too many uniques), pymnt_plan (all the same), title (too many uniques, seems to have been reconsolidated under purpose, don't use address zip code or state for now

In [47]:
for i in obj_cols:
    df = df.merge(pd.get_dummies(df[i]
                                 , prefix='{0}'.format(i))
                , left_index=True
                , right_index=True)

In [48]:
df.drop(obj_cols, axis=1, inplace=True)

In [49]:
df.head()

,loan_amnt,term,int_rate,installment,emp_length,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,...,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding
0,10400,36,6.99,321.08,8.0,58000.0,14.92,0,2,42.0,...,0,0,0,0,0,0,0,0,0,0
1,15000,60,12.39,336.64,10.0,78000.0,12.03,0,0,NaN,...,0,0,0,0,0,0,0,0,0,0
2,21425,60,15.59,516.36,6.0,63800.0,18.49,0,0,60.0,...,0,0,0,0,0,0,0,0,0,0
3,7650,36,13.66,260.20,0.0,50000.0,34.81,0,1,NaN,...,0,0,0,0,0,0,0,0,0,0
5,9600,36,13.66,326.53,10.0,69000.0,25.81,0,0,NaN,...,0,0,0,0,0,0,0,0,0,0


In [50]:
def bin_nulls(col, bins):
    bin_num = []
    for func in bins:
        bin_num.append(func(col))
    bin_num = list(set(bin_num))
    bin_num.sort()
    col = pd.cut(col, bin_num, right=True, include_lowest=True)
    dum = pd.get_dummies(col, dummy_na=True)
    return dum

In [51]:
for col in df.columns[df.isnull().any()]:
    df[col] = bin_nulls(df[col], bins=[np.min, partial(np.nanpercentile, q=25), np.nanmedian, partial(np.nanpercentile, q=75), np.max])

In [52]:
np.sum(df.isnull().any())

0

In [53]:
df.to_csv('2014_clean_1.csv')

### Modeling

In [68]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.grid_search import GridSearchCV

In [62]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

In [55]:
rand = 128

In [56]:
y = df.pop('target')
X = df

In [57]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3)

In [80]:
rfc = RandomForestClassifier(n_estimators=250, max_features='sqrt')

In [81]:
params = dict(n_estimators=[200, 500, 1000]
             , max_features=['sqrt', 'log2']
             , max_depth=[10, 25, None])

In [82]:
gc = GridSearchCV(rfc, params)

In [ ]:
gc.fit(X_train, y_train)

In [76]:
gc.score(X_train, y_train)

AttributeError: 'GridSearchCV' object has no attribute 'scorer_'

In [61]:
rfc.score(X_test, y_test)

0.7403265710044532

In [77]:
y_true = y_test
y_pred = gc.predict(X_test)

AttributeError: 'GridSearchCV' object has no attribute 'best_estimator_'

In [64]:
roc_auc_score(y_true, y_pred)

0.5386784891881008

In [67]:
print (classification_report(y_true, y_pred))

             precision    recall  f1-score   support

          0       0.57      0.11      0.18     13455
          1       0.75      0.97      0.85     37070

avg / total       0.70      0.74      0.67     50525

